### 사용자 요청별 검색 및 요약 전략 맞춤 설계 구현

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

In [3]:
# 🔸 LangChain v1.0 - 사용자 요청별 검색/요약 전략 맞춤 설계 에이전트
# pip install langchain langchain-openai langchain-community faiss-cpu python-dotenv

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool
from langchain.agents import create_agent

load_dotenv()

# -------------------------------------------------------
# 1) 기본 설정
# -------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# -------------------------------------------------------
# 2) 샘플 문서 데이터 생성
# -------------------------------------------------------
docs = [
    Document(page_content="RAG는 검색 결과를 생성 모델과 결합하여 정확한 응답을 생성한다."),
    Document(page_content="Fusion RAG는 여러 검색 쿼리의 결과를 결합해 품질을 향상시킨다."),
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 구성하기 위한 파이썬 프레임워크이다."),
    Document(page_content="LangGraph는 LangChain 워크플로우를 시각적으로 설계하기 위한 도구이다."),
    Document(page_content="OpenAI의 GPT-4o-mini는 효율적인 멀티모달 추론이 가능한 최신 모델이다."),
]
vectorstore = FAISS.from_documents(docs, embeddings)

# -------------------------------------------------------
# 3) 검색용 도구 정의
# -------------------------------------------------------
@tool
def search_docs(query: str) -> str:
    """간단한 정보 검색용 도구 (핵심 문장 반환)."""
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    results = retriever.invoke(query)
    text = "\n".join([r.page_content for r in results])
    return f"[검색 결과]\n{text}"

# -------------------------------------------------------
# 4) 요약용 도구 정의
# -------------------------------------------------------
@tool
def summarize_docs(topic: str) -> str:
    """관련 문서를 종합 요약."""
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    results = retriever.invoke(topic)
    joined = "\n".join([r.page_content for r in results])
    prompt = f"""
    아래 문서 내용을 기반으로 '{topic}'에 대한 핵심 요약을 작성하라.
    {joined}
    """
    response = llm.invoke(prompt)
    return response.content

# -------------------------------------------------------
# 5) 비교용 도구 정의
# -------------------------------------------------------
@tool
def compare_docs(item_a: str, item_b: str) -> str:
    """두 개념을 비교·분석한다."""
    prompt = f"""
    다음 두 개념을 비교하여 차이점과 공통점을 명확히 정리하라.
    - A: {item_a}
    - B: {item_b}
    """
    response = llm.invoke(prompt)
    return response.content

# -------------------------------------------------------
# 6) 에이전트 생성
# -------------------------------------------------------
agent = create_agent(
    model=llm,
    tools=[search_docs, summarize_docs, compare_docs],
    system_prompt=(
        "당신은 사용자의 요청을 분석해 가장 적절한 전략을 선택하는 AI 연구 조교이다.\n"
        "- 요청이 간단한 정의·정보면 search_docs 사용\n"
        "- 개념 정리나 분석이면 summarize_docs 사용\n"
        "- 비교나 차이점 질문이면 compare_docs 사용\n"
        "각 도구를 필요시 호출하여 최종 답변을 완성하라."
    ),
)

# -------------------------------------------------------
# 7) 실행 예시
# -------------------------------------------------------
queries = [
    "RAG Fusion이 뭐야?",
    "LangChain과 LangGraph의 차이를 알려줘.",
    "OpenAI의 GPT-4o-mini와 관련된 핵심 내용을 요약해줘."
]

for q in queries:
    print(f"\n[사용자 요청] {q}")
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": q}
        ]
    })
    print("[응답 결과]")
    print(result["messages"][-1].content)



[사용자 요청] RAG Fusion이 뭐야?
[응답 결과]
RAG Fusion은 여러 검색 쿼리의 결과를 결합하여 품질을 향상시키는 방법입니다. RAG는 검색 결과를 생성 모델과 결합하여 보다 정확한 응답을 생성하는 기술입니다.

[사용자 요청] LangChain과 LangGraph의 차이를 알려줘.
[응답 결과]
LangChain과 LangGraph는 자연어 처리(NLP) 및 인공지능(AI) 분야에서 사용되는 도구이지만, 그 목적과 기능에서 차이가 있습니다.

### 차이점

1. **목적 및 기능**
   - **LangChain**: 언어 모델을 활용하여 대화형 AI, 텍스트 생성, 정보 검색 등 다양한 애플리케이션을 구축하는 데 중점을 둡니다.
   - **LangGraph**: 언어 모델과 관련된 데이터 및 관계를 시각적으로 표현하고 분석하는 데 중점을 두며, 데이터 간의 관계를 이해하고 복잡한 정보 구조를 시각화하는 데 유용합니다.

2. **사용 사례**
   - **LangChain**: 챗봇, 자동화된 콘텐츠 생성, 질문 응답 시스템 등 다양한 AI 기반 애플리케이션에서 사용됩니다.
   - **LangGraph**: 데이터 분석, 관계 맵 작성, 정보 시각화 등 데이터 중심의 작업에서 주로 사용됩니다.

3. **구조 및 접근 방식**
   - **LangChain**: 모듈화된 구조를 가지고 있어 다양한 언어 모델과 통합하여 사용할 수 있으며, API를 통해 쉽게 접근할 수 있습니다.
   - **LangGraph**: 그래프 기반의 구조를 통해 데이터 간의 관계를 시각적으로 표현하며, 데이터 분석 도구와의 통합이 강조됩니다.

### 공통점

1. **AI 및 NLP 활용**: 두 개념 모두 자연어 처리 및 인공지능 기술을 활용하여 정보를 처리하고 분석하는 데 중점을 둡니다.
2. **모듈화 및 확장성**: LangChain과 LangGraph 모두 모듈화된 구조를 가지고 있어, 사용자가 필요에 따라 기능을 추가하거나 변경할 수 있는 확